# Convolutional Neural Network

### Importing the libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
tf.__version__

'2.18.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [ ]:
# Ініціалізація ImageDataGenerator для підготовки зображень з різними трансформаціями
train_datagen = ImageDataGenerator(
    rescale = 1./255,         # Масштабування пікселів зображень до діапазону [0, 1]
    shear_range = 0.2,        # Рандомне зсування (shear) зображень для збільшення варіативності
    zoom_range = 0.2,         # Рандомне збільшення/зменшення зображень (zoom) для варіативності
    horizontal_flip = True    # Рандомне горизонтальне відображення зображень
)

# Завантаження зображень з директорії 'dataset/training_set' з певними параметрами
training_set = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Gen AI Cource/Урок 4: Розширені архітектури нейронних мереж/Convolutional Neural Networks (CNN)/Python/dataset/training_set',     # Шлях до директорії з тренувальними зображеннями
    target_size = (64, 64),     # Перемасштабування зображень до розміру 64x64 пікселів
    batch_size = 32,            # Розмір пакету (batch) зображень, який буде використовуватись при тренуванні
    class_mode = 'binary'       # Модель кодування міток: двійкова класифікація (наприклад, 0 або 1)
)


Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [ ]:
test_datagen = ImageDataGenerator(
    rescale = 1./255  # Масштабування пікселів зображень до діапазону [0, 1] для нормалізації
)

# Завантаження тестових зображень з директорії 'dataset/test_set' з певними параметрами
test_set = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Gen AI Cource/Урок 4: Розширені архітектури нейронних мереж/Convolutional Neural Networks (CNN)/Python/dataset/test_set',     # Шлях до директорії з тестовими зображеннями
    target_size = (64, 64),  # Перемасштабування зображень до розміру 64x64 пікселів
    batch_size = 32,         # Розмір пакету (batch) зображень, який буде використовуватись при тестуванні
    class_mode = 'binary'    # Мод аль кодування міток: двійкова класифікація (наприклад, 0 або 1)
)


Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
# Створення порожньої послідовної моделі (Sequential) для побудови нейронної мережі
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [ ]:
# Додавання 2D-конвуляційного шару до моделі
cnn.add(tf.keras.layers.Conv2D(
    filters=32,               # Кількість фільтрів (каналів) у шарі, що визначає кількість вихідних характеристик
    kernel_size=3,            # Розмір ядра (фільтру), який буде застосовуватись до зображення (3x3)
    activation='relu',        # Функція активації ReLU для додавання нелінійності
    input_shape=[64, 64, 3]   # Формат вхідних даних: зображення розміру 64x64 з 3 каналами (RGB)
))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [ ]:
# Додавання шару максимального підвибору (MaxPooling) для зменшення розміру зображення
cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=2,        # Розмір пулінгового вікна (2x2) для вибору максимального значення в кожному регіоні
    strides=2           # Крок скидання вікна, який визначає, на скільки пікселів переміщається вікно
))


### Adding a second convolutional layer

In [ ]:
# Додавання ще одного 2D-конвуляційного шару до моделі
cnn.add(tf.keras.layers.Conv2D(
    filters=32,               # Кількість фільтрів (каналів) у шарі, що визначає кількість вихідних характеристик
    kernel_size=3,            # Розмір ядра (фільтру), який буде застосовуватись до зображення (3x3)
    activation='relu'         # Функція активації ReLU для додавання нелінійності
))

# Додавання ще одного шару максимального підвибору (MaxPooling) для зменшення розміру зображення
cnn.add(tf.keras.layers.MaxPool2D(
    pool_size=2,        # Розмір пулінгового вікна (2x2) для вибору максимального значення в кожному регіоні
    strides=2           # Крок скидання вікна, який визначає, на скільки пікселів переміщається вікно
))


### Step 3 - Flattening

In [ ]:
# Додавання шару Flatten для перетворення 2D-матриць в 1D-вектор
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [ ]:
# Додавання повнозв'язного (Dense) шару з 128 одиницями та функцією активації ReLU
cnn.add(tf.keras.layers.Dense(
    units=128,             # Кількість нейронів (одиниць) у цьому шарі
    activation='relu'      # Функція активації ReLU для додавання нелінійності
))


### Step 5 - Output Layer

In [ ]:
# Додавання вихідного повнозв'язного (Dense) шару з 1 одиницею та функцією активації Sigmoid
cnn.add(tf.keras.layers.Dense(
    units=1,               # Кількість нейронів (1) у вихідному шарі (для бінарної класифікації)
    activation='sigmoid'   # Функція активації Sigmoid, яка обмежує вихідне значення до діапазону [0, 1]
))


## Part 3 - Training the CNN

### Compiling the CNN

In [ ]:
# Компіляція моделі з вказаними оптимізатором, функцією втрат і метриками
cnn.compile(
    optimizer='adam',                  # Оптимізатор Adam для ефективного оновлення ваг моделі
    loss='binary_crossentropy',         # Функція втрат для бінарної класифікації (перехресна ентропія)
    metrics=['accuracy']                # Метрика точності для оцінки ефективності моделі
)

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Оголошуємо колбек EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_loss',          # Моніторинг метрики для зупинки (можна використовувати 'val_accuracy' для точності)
    patience=3,                  # Кількість епох, після яких зупиняємо тренування, якщо немає покращень
    verbose=1,                   # Виведення повідомлення про зупинку тренування
    restore_best_weights=True    # Повертаємо ваги моделі на найкращий етап (найменший val_loss)
)

# Навчання моделі з використанням early stopping
cnn.fit(
    x=training_set,              # Тренувальні дані
    validation_data=test_set,    # Дані для валідації
    epochs=5,                   # Кількість епох для тренування
    steps_per_epoch=len(training_set),  # Кількість кроків на одну епоху
    validation_steps=len(test_set),    # Кількість кроків на валідацію
    verbose=1,                   # Виведення прогресу тренування
    callbacks=[early_stopping]    # Додаємо колбек EarlyStopping для моніторингу процесу
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 2272s 9s/step - accuracy: 0.5574 - loss: 0.6869 - val_accuracy: 0.6755 - val_loss: 0.6090
Epoch 2/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 77s 186ms/step - accuracy: 0.6749 - loss: 0.5980 - val_accuracy: 0.7185 - val_loss: 0.5555
Epoch 3/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 46s 183ms/step - accuracy: 0.7119 - loss: 0.5596 - val_accuracy: 0.7410 - val_loss: 0.5492
Epoch 4/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 82s 185ms/step - accuracy: 0.7338 - loss: 0.5316 - val_accuracy: 0.7610 - val_loss: 0.5043
Epoch 5/5
250/250 ━━━━━━━━━━━━━━━━━━━━ 84s 195ms/step - accuracy: 0.7519 - loss: 0.5048 - val_accuracy: 0.7320 - val_loss: 0.5430
Restoring model weights from the end of the best epoch: 4.


## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
# Завантаження зображення для передбачення та зміна його розміру до 64x64 пікселів
test_image = image.load_img('/content/drive/MyDrive/Gen AI Cource/Урок 4: Розширені архітектури нейронних мереж/Convolutional Neural Networks (CNN)/Python/yorkshire-terrier.jpg', target_size = (64, 64))

# Перетворення зображення в масив (числове представлення)
test_image = image.img_to_array(test_image)

# Додавання додаткового виміру, щоб відповідати формату вхідних даних моделі (1, 64, 64, 3)
test_image = np.expand_dims(test_image, axis = 0)

# Прогнозування класу для зображення
result = cnn.predict(test_image)

# Перевірка, до якого класу належить зображення: 'dog' або 'cat'
training_set.class_indices   # Виведення відповідностей класів (мітки) для тренувального набору
if result[0][0] >= 0.5:
    prediction = 'dog'
else:
    prediction = 'cat'

print(prediction)

probability = result[0][0]
print(f"Probability: {probability}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
cat
Probability: 0.24745285511016846


In [ ]:
print(prediction)